In [1]:
from pyEDM import *
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import fiona
import rasterio.mask
from descartes import PolygonPatch
from rasterio.plot import show
from tqdm import tqdm
import datetime
import random
from sklearn.preprocessing import StandardScaler

In [138]:
# precip_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly2/'
# temp_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly2/'
# ndvi_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly2/'

In [9]:
precip_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/EA/'
temp_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/EA/'
ndvi_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EA/'
et_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/ETos_p05_EA/'
sm10_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA10/'
sm40_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA40/'
sm100_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA100/'
sm200_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA200/'

In [3]:
def generate_dekads(start, end):
    '''
    Generates list of timestamps of dekads between two dates
    
    Parameters
    ----------
    start : pd.datetime object
        Start date of list
    end : pd.datetime object
        end date of list
    '''
    
    dtrange = pd.date_range(start, end)
    days = list(range(len(dtrange))) #length of dtrange
    daysDF = pd.DataFrame({'Days': days}, index=dtrange)
    d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 
    dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
    dekads = daysDF.groupby(dekaddates).mean()
    dekads = dekads.index
    
    return dekads

In [4]:
start = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)

dekads = generate_dekads(start, end)

In [5]:
def stack_rasters(in_dir, keyword):
    
    '''
    This function creates a raster stack xarray DataArray given a specified variable keyword.
    Keywords = 'precip', 'ndvi','lst'
    
    '''
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    stacked = []
    
    for filename in tqdm(files): 
        if keyword in filename:
    
            file = xr.open_rasterio(in_dir+filename)
            stacked.append(file.sel(band=1))
            
    combined = xr.concat(stacked, dim = 'time')
  

    return combined.assign_coords(time=dekads)

In [59]:
stack_out_temp = stack_rasters(temp_ea, 'temp')

100%|██████████| 606/606 [00:02<00:00, 245.10it/s]


In [119]:
stack_out_precip = stack_rasters(precip_ea, 'precip')

100%|██████████| 606/606 [00:02<00:00, 236.13it/s]


In [8]:
stack_out_ndvi = stack_rasters(ndvi_ea, 'ndvi')

100%|██████████| 606/606 [00:09<00:00, 66.36it/s]


In [146]:
stack_out_et = stack_rasters(et_ea, 'ETos')

100%|██████████| 606/606 [00:03<00:00, 189.59it/s]


In [10]:
stack_out_sm10 = stack_rasters(sm10_ea, 'SM')

100%|██████████| 606/606 [00:03<00:00, 194.44it/s]


In [11]:
stack_out_sm40 = stack_rasters(sm40_ea, 'SM')

100%|██████████| 606/606 [00:02<00:00, 288.19it/s]


In [12]:
stack_out_sm100 = stack_rasters(sm100_ea, 'SM')

100%|██████████| 606/606 [00:02<00:00, 208.67it/s]


In [13]:
stack_out_sm200 = stack_rasters(sm200_ea, 'SM')

100%|██████████| 606/606 [00:02<00:00, 211.25it/s]


In [60]:
#stack_out_ndvi.groupby(.strftime('%m-%d')

In [68]:
ds_anom_precip_clim = stack_out_precip.groupby('time.month') - stack_out_precip.groupby("time.month").mean("time")

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [25]:
ds_ndvi = (stack_out_ndvi-100)/100
ds_anom_ndvi_clim = ds_ndvi.groupby('time.month') - ds_ndvi.groupby('time.month').mean('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [72]:
ds_anom_temp_clim = stack_out_temp.groupby('time.month') - stack_out_temp.groupby("time.month").mean("time")

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [148]:
ds_anom_et_clim = stack_out_et.groupby('time.month') - stack_out_et.groupby('time.month').mean('time')

In [14]:
ds_anom_sm10_clim = stack_out_sm10.groupby('time.month') - stack_out_sm10.groupby('time.month').mean('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [15]:
ds_anom_sm40_clim = stack_out_sm10.groupby('time.month') - stack_out_sm40.groupby('time.month').mean('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [16]:
ds_anom_sm100_clim = stack_out_sm10.groupby('time.month') - stack_out_sm100.groupby('time.month').mean('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [17]:
ds_anom_sm200_clim = stack_out_sm10.groupby('time.month') - stack_out_sm200.groupby('time.month').mean('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [77]:
ds_anom_ndvi_norm = (ds_ndvi - ds_ndvi.mean('time'))/ds_ndvi.std('time')

In [80]:
#get first value
#ds_ndvi[0,0][0].values

In [120]:
ds_anom_precip_norm = (stack_out_precip - stack_out_precip.mean('time'))/stack_out_precip.std('time')

In [82]:
ds_anom_temp_norm = (stack_out_temp - stack_out_temp.mean('time'))/stack_out_temp.std('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [150]:
ds_anom_et_norm = (stack_out_et - stack_out_et.mean('time'))/stack_out_et.std('time')

In [18]:
ds_anom_sm10_norm = (stack_out_sm10 - stack_out_sm10.mean('time'))/stack_out_sm10.std('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [19]:
ds_anom_sm40_norm = (stack_out_sm10 - stack_out_sm40.mean('time'))/stack_out_sm40.std('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [20]:
ds_anom_sm100_norm = (stack_out_sm10 - stack_out_sm100.mean('time'))/stack_out_sm100.std('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [21]:
ds_anom_sm200_norm = (stack_out_sm10 - stack_out_sm200.mean('time'))/stack_out_sm200.std('time')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [83]:
ds_anom_ndvi_demean = (ds_ndvi - ds_ndvi.mean('time'))

In [84]:
ds_anom_precip_demean = (stack_out_precip - stack_out_precip.mean('time'))

In [85]:
ds_anom_temp_demean = (stack_out_temp - stack_out_temp.mean('time'))

In [152]:
ds_anom_et_demean = (stack_out_et - stack_out_et.mean('time'))

In [22]:
ds_anom_sm10_demean = (stack_out_sm10 - stack_out_sm10.mean('time'))

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [23]:
ds_anom_sm40_demean = (stack_out_sm40 - stack_out_sm40.mean('time'))

In [24]:
ds_anom_sm100_demean = (stack_out_sm10 - stack_out_sm100.mean('time'))

In [25]:
ds_anom_sm200_demean = (stack_out_sm10 - stack_out_sm200.mean('time'))

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [28]:
def dataarray2tiff(dataarray, in_dir, out_dir, out_name):
    
    '''
    This function converts a 2D xarray DataArray into a tif using the profile of an existing tif

    '''
    

    with rasterio.open(in_dir+ os.listdir(in_dir)[0]) as src:
        profile = src.profile
        
    for i in np.arange(0,len(os.listdir(in_dir))):
        data = np.array(dataarray[i])
        
       
        with rasterio.open(out_dir + out_name + str(np.array(dataarray[i].time))[:10] + '.tif', 'w', **profile) as dst:
            
            data = data.astype(np.float32)
            dst.write(data, indexes = 1)

In [ ]:
dataarray2tiff(ds_anom_ndvi_clim, ndvi_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Clim/', 'anom_ea_ndvi_')

In [ ]:
dataarray2tiff(ds_anom_ndvi_norm, ndvi_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Norm/', 'anom_ea_ndvi_')

In [ ]:
dataarray2tiff(ds_anom_ndvi_demean, ndvi_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Demean/', 'anom_ea_ndvi_')

In [87]:
dataarray2tiff(ds_anom_precip_clim, precip_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Clim/', 'anom_ea_precip_')

In [126]:
dataarray2tiff(ds_anom_precip_norm, precip_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Norm/', 'anom_ea_precip_')

In [125]:
dataarray2tiff(ds_anom_precip_demean, precip_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Demean/', 'anom_ea_precip_')

In [ ]:
dataarray2tiff(ds_anom_temp_clim, temp_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Clim/', 'anom_ea_temp_')

In [ ]:
dataarray2tiff(ds_anom_temp_norm, temp_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Norm/', 'anom_ea_temp_')

In [ ]:
dataarray2tiff(ds_anom_temp_demean, temp_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Demean/', 'anom_ea_temp_')

In [ ]:
dataarray2tiff(ds_anom_et_clim, et_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Anomaly_Clim/', 'anom_ea_et_')

In [ ]:
dataarray2tiff(ds_anom_et_norm, et_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Anomaly_Norm/', 'anom_ea_et_')

In [ ]:
dataarray2tiff(ds_anom_et_demean, et_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Anomaly_Demean/', 'anom_ea_et_')

In [29]:
dataarray2tiff(ds_anom_sm10_clim, sm10_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly10_Clim/', 'anom_ea_sm10_')

In [ ]:
dataarray2tiff(ds_anom_sm40_clim, sm40_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly40_Clim/', 'anom_ea_sm40_')

In [32]:
dataarray2tiff(ds_anom_sm100_clim, sm100_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly100_Clim/', 'anom_ea_sm100_')

In [33]:
dataarray2tiff(ds_anom_sm200_clim, sm200_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly200_Clim/', 'anom_ea_sm200_')

In [34]:
dataarray2tiff(ds_anom_sm10_norm, sm10_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly10_Norm/', 'anom_ea_sm10_')

In [ ]:
dataarray2tiff(ds_anom_sm40_norm, sm40_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly40_Norm/', 'anom_ea_sm40_')

In [37]:
dataarray2tiff(ds_anom_sm100_norm, sm100_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly100_Norm/', 'anom_ea_sm100_')

In [38]:
dataarray2tiff(ds_anom_sm200_norm, sm200_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly200_Norm/', 'anom_ea_sm200_')

In [39]:
dataarray2tiff(ds_anom_sm10_demean, sm10_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly10_Demean/', 'anom_ea_sm10_')

In [ ]:
dataarray2tiff(ds_anom_sm40_demean, sm40_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly40_Demean/', 'anom_ea_sm40_')

In [41]:
dataarray2tiff(ds_anom_sm100_demean, sm100_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly100_Demean/', 'anom_ea_sm100_')

In [42]:
dataarray2tiff(ds_anom_sm200_demean, sm200_ea, '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly200_Demean/', 'anom_ea_sm200_')

In [12]:
def pixelwise_ts_table(in_dir, keyword):
    
    '''
    This function searches through the file directory and creates a dataframe of pixel values over time from rasters
    based on a given keyword describing the environmental variable of interest to generate a raster time series stack 
    
   In the output dataframe, columns are each pixel extracted from the raster (read squentially into a list)
   and rows are the same pixel over time (dekads going down)
    
    Args:
    - in_dir: path to the input directory
    -keyword: string that is unique to the environmental variable
        options: ['precip', 'temp', 'ndvi']
    '''
    
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains(keyword)
    files = files[tifs]
    #n_samples = len(files)
    
    
    pixelwise_TS = []
    
    for filename in tqdm(files): 
        
        
            open_file = xr.open_rasterio(in_dir+filename).sel(band=1)
            array = open_file.values
            #
            pixel_list = array.ravel().tolist()
            
            pixelwise_TS.append(pixel_list)

        
    return pd.DataFrame(pixelwise_TS)

    

In [33]:
ndvi_table_anom1 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly/', 'ndvi')

100%|██████████| 606/606 [00:08<00:00, 72.80it/s]


In [48]:
ndvi_table_anom2 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly2/', 'ndvi')

100%|██████████| 606/606 [00:07<00:00, 81.37it/s]


In [52]:
ndvi_table_anom3 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly3/', 'ndvi')

100%|██████████| 606/606 [00:07<00:00, 82.98it/s]


In [93]:
precip_table_anom1 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly/', 'precip')

100%|██████████| 606/606 [00:08<00:00, 73.47it/s]


In [95]:
precip_table_anom2 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly2/', 'precip')

100%|██████████| 606/606 [00:07<00:00, 77.93it/s]


In [97]:
precip_table_anom3 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly3/', 'precip')

100%|██████████| 606/606 [00:14<00:00, 42.93it/s]


In [99]:
precip_table_clim = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Clim/', 'precip')

100%|██████████| 606/606 [00:08<00:00, 71.86it/s]


In [130]:
precip_table_norm = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Norm/', 'precip')

100%|██████████| 606/606 [00:07<00:00, 79.01it/s]


In [103]:
precip_table_demean = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly_Demean/', 'precip')

100%|██████████| 606/606 [00:07<00:00, 76.49it/s]


In [105]:
temp_table_anom1 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly/', 'temp')

100%|██████████| 606/606 [00:08<00:00, 72.93it/s]


In [107]:
temp_table_anom2 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly2/', 'temp')

100%|██████████| 606/606 [00:07<00:00, 76.44it/s]


In [109]:
temp_table_anom3 = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly3/', 'temp')

100%|██████████| 606/606 [00:14<00:00, 40.53it/s]


In [110]:
temp_anom_clim = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Clim/', 'temp')

100%|██████████| 606/606 [00:08<00:00, 70.55it/s]


In [112]:
temp_anom_norm = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Norm/', 'temp')

100%|██████████| 606/606 [00:08<00:00, 74.49it/s]


In [114]:
temp_anom_demean = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly_Demean/', 'temp')

100%|██████████| 606/606 [00:08<00:00, 74.15it/s]


In [31]:
ndvi_table_anom1_clim = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Clim/', 'ndvi')

100%|██████████| 606/606 [00:07<00:00, 77.69it/s]


In [54]:
ndvi_table_anom_demean = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Demean/', 'ndvi')

100%|██████████| 606/606 [00:08<00:00, 75.28it/s]


In [128]:
ndvi_table_anom_norm = pixelwise_ts_table('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Norm/', 'ndvi')

100%|██████████| 606/606 [00:07<00:00, 79.34it/s]
